# Bayesian model fitting with *emcee*
### [Giuliano Antoniciello](http://www.dfa.unipd.it/index.php?id=1744)
### Py@stro - April 24th, 2018

The [** *emcee* **](http://dfm.io/emcee/current/) MCMC sampler is a powerful tool to perform complex and computationally daunting Bayesian inference.

We will review the fundamentals of Bayesian probability theory and apply them to some model fitting examples in Astronomy. The needed Python libraries are:

1. [*emcee*](http://dfm.io/emcee/current/) to run the MCMC simulations;
2. [*corner*](https://corner.readthedocs.io/en/latest/) to show the results.

To run this notebook you'll need also the following Python libraries:

1. [*Numpy*](http://www.numpy.org/)
2. [*Matplotlib*](https://matplotlib.org/)
3. [*Seaborn*](https://seaborn.pydata.org/) (optional)

**WARNING: this notebook is intended for a Python 2.7 kernel!**

## The linear model
#### Let's generate a synthetic dataset
First we draw some data points from our parametric model and add a Gaussian noise.
Our aim is to determine the most probable values for the model parameters. In this simple example the target function is just a linear relation with two parameters:

$f(x) = mx+q$

In [ ]:
import numpy as np

# Set the number of data points
n_points = 25

# Set the true values of model parameters
m_true = 5.0
q_true = -3.0

# Set the pointwise uncertainty
sigma = 5.0

# Generate the synthetic dataset
x = np.sort(10*np.random.random(size=n_points))
y_true = m_true*x + q_true

# Add Gaussian noise and error bars
y = y_true + np.random.normal(loc=0.,scale=sigma, size=n_points)
yerr = sigma*np.ones(n_points)

# Compute the reduced chi square and the initial log-likelihood
num = (y_true-y)*(y_true-y)
den = yerr*yerr
chi_square = np.sum(np.true_divide(num, den))
reduced_chi_square = chi_square/(n_points - 2)

# log(error)
log_err = np.sum(np.log(yerr))

# log_likelihood value
true_log_likelihood = -(0.5*n_points*np.log(2*np.pi)) - log_err - (0.5 * chi_square)
print "Reduced chi square: " + str(reduced_chi_square)
print "Log-likelihood: " + str(true_log_likelihood)

#### Plot the synthetic dataset

In [ ]:
# seaborn-style plot
# fancy plots :)
import seaborn as sns
sns.set(style='ticks')
sns.set_color_codes()

# Plot the data points and the target function
import matplotlib.pyplot as plt
plt.errorbar(x,y,yerr,fmt='o',color='r')
plt.plot(x,y_true, color='b', alpha=0.7)
plt.xlabel('$x$')
plt.ylabel('$f(x)$')
plt.show()

#### Model function
Here we define a function that takes an $x$ value and return the corresponding $f(x)$.

In [ ]:
# A function for our linear model (as suggested by prior information)
def linear_model(x, m, q):
    """
    Linear model for the target function

    Input:
    -----

    - x, independet variable (some units)
    - m, slope
    - q, constant term

    Output:
    ------

    - f(x), the value of target function
    """
    
    # Plain and simple: a linear model
    model = m*x + q
    
    return model

#### (log) prior probability
Notice that *emcee* always uses log-probabilities! 
Here the log-prior probability takes data and model parameters' values $\theta$ and returns a log-probability:

$lnprior = p(\theta \mid DX)$

In [ ]:
# Function for uniform prior probabilities
def lnprior(theta, x_vector, y_vector, yerr_vector, m_range, q_range):
    
    """
    Log-prior probability function

    Input:
    -----

    - theta, the model parameters (m, q)
    - x_vector, array of independet variable values (some units)
    - y_vector, array of f(x) values
    - yerr_vector, array of uncertainties for the f(x) value
    - m_range, tuple with lower and upper limits for the prior interval of m
    - q_range, tuple with lower and upper limits for the prior interval of q

    Output:
    ------

    - log(prior(theta)), logarithm of prior probability for the set of value parameters theta
    """
    
    import numpy as np
    
    # Unpack the parameters
    m, q = theta

    # Uninformative prior for m
    #--------------------------
    # Extract the interval's limits
    m_min, m_max = m_range
    # Compute the interval's lenght
    delta_m = m_max - m_min
    # Condition for the uniform prior
    m_cond = m_min < m < m_max
    
    # Uninformative prior for q
    #--------------------------
    # Extract the interval's limits
    q_min, q_max = q_range
    # Compute the interval's lenght
    delta_q = q_max - q_min
    # Condition for the uniform prior
    q_cond = q_min < m < q_max

    # Global condition for the priors
    cond = m_cond and q_cond
    
    # Evaluate the condition
    if cond:
        return -np.log(delta_m * delta_q)
    else:
        return -np.inf

#### (log) likelihood
The likelihood function, from the point-of-view of a fixed dataset, is not a probability, since it's not normalized to unit. It a function of model parameters expressing "how liklely" is a given set of parameter values once we observe the data $D$ and have the prior information $X$. Indeed, the same term (likelihood) can be interpreted as a probability if we consider a fixed set of parameter values: in this case, likelihood is the sampling distribution from which data points are drawn. Since collecting data is typically a result of many complex operation, we communly assume a Gaussian likelihood for statistically independent measures:

$likelihood = p(D \mid \theta X) = \prod_{i=1}^{n_{points}} \dfrac{1}{\sqrt{2 \pi} \sigma_i} \exp{-\dfrac{(x_i - f(x_i))^2}{2 \sigma_i^2}}$

In [ ]:
# function for the natural logarithm of the likelihood
def lnlike(theta, x_vector, y_vector, yerr_vector, m_range, q_range):
    
    """
    Log-prior probability function

    Input:
    -----

    - theta, the model parameters (m, q)
    - x_vector, array of independet variable values (some units)
    - y_vector, array of f(x) values
    - yerr_vector, array of uncertainties for the f(x) value
    - m_range, tuple with lower and upper limits for the prior interval of m
    - q_range, tuple with lower and upper limits for the prior interval of q

    Output:
    ------

    - log(likelihood(theta)), logarithm of the likelihood function for the set of value parameters theta
    """
    
    #---------------------------------------

    import numpy as np

    # Extract the parameter values
    m, q = theta
    
    # Get the model values vector
    model_vector = np.zeros(x_vector.shape[0])
    for index, x in enumerate(x_vector):
        model_vector[index] = linear_model(x, m, q)

    # Compute the chi^2
    num = (y_vector - model_vector) * (y_vector - model_vector)
    den = yerr_vector * yerr_vector
    chi_squared = np.sum(np.true_divide(num, den))

    # Number of data points
    n_points = x_vector.shape[0]

    # Compute the sum of log(uncertainties)
    log_err = np.sum(np.log(yerr_vector))

    # Compute the log_likelihood value
    log_likelihood = -(0.5*n_points*np.log(2*np.pi)) - log_err - (0.5 * chi_squared)

    #---------------------------------------

    return log_likelihood

#### (log) posterior probability
Log-posterior probability is simply

$log(posterior) = log(prior) + log(likelihood)$

since, from the Bayes theorem

$posterior \propto prior \times likelihood$

In [ ]:
def lnprob(theta, x_vector, y_vector, yerr_vector, m_range, q_range):
    
    """
    Log-posterior probability function

    Input:
    -----

    - theta, the model parameters (m, q)
    - x_vector, array of independet variable values (some units)
    - y_vector, array of f(x) values
    - yerr_vector, array of uncertainties for the f(x) value
    - m_range, tuple with lower and upper limits for the prior interval of m
    - q_range, tuple with lower and upper limits for the prior interval of q

    Output:
    ------

    - log(posterior(theta)), logarithm of the likelihood function for the set of value parameters theta
    """
    
    import numpy as np

    # Compute prior probability for the current walker position
    lp = lnprior(theta, x_vector, y_vector, yerr_vector, m_range, q_range)

    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike(theta, x_vector, y_vector, yerr_vector, m_range, q_range)

#### Initialize the MCMC sampler
##### Affine invariant sampler by [*Goodman & Weare (2010)*](https://msp.org/camcos/2010/5-1/p04.xhtml)
This algorithm has been design to overcome the difficulties that standard MCMC algorithms (such as Metropolis-Hastings) encounter in treating highly non linear models and (possibly strongly) correlated parameters. See the reference for mathematical details.

In [ ]:
# Define the n. of walkers and n. of parameters (--> dimension of the parameter space)
n_dim = 2
n_walkers = 50

In [ ]:
# Set the number of iteration to be performed
N_iter = 2000
# Set the burn-in steps
burn_in = 1000

In [ ]:
# Dataset as arguments of the MCMC simulation
m_range = (0.,10.)
q_range = (-10.,10.)
args = (x, y, yerr, m_range, q_range)

# New object from the emcee class
# Instance for the MCMC simulation
import emcee
sampler = emcee.EnsembleSampler(nwalkers=n_walkers, dim=n_dim, lnpostfn=lnprob, a=2.0, args=args, threads=8)

In [ ]:
# Set the seed for the Mersenne-Twister random number generatore
np.random.seed(42)
# Inizialize the walkers in a small hyper-sphere
p0 = np.array([[m_true, q_true] + (10e-4)*np.random.normal(loc=0.0, scale=1.0, size=2) \
               for i in xrange(n_walkers)], dtype=float)

In [ ]:
# Instance for the sample generator
samples = sampler.sample(p0=p0, rstate0=52, iterations=N_iter, thin=1, storechain=False)

In [ ]:
# Initialize the 3-D matrices for the walker positions at each step (chains) and the logposterior values (posteriors)
# Each slice of these 3-D matrices corresponds to a single iteration step
# with walkers on the rows and parameters on the columns
chains = np.zeros((n_walkers, n_dim, N_iter))
posteriors = np.zeros((n_walkers, 1, N_iter))

#### Let's run the MCMC simualtion!
So far we've just instantiated the *emcee* object and the sample generator. Now we go through the computationally expensive part of our simulation: drawing and storing the samples from the posterior pdfs.

In [ ]:
%%time
# Get the posterior values from the sample generator
for index, sample in enumerate(samples):
    # Positions of walkers in the parameter space
    positions_matrix = sample[0]
    chains[:, :, index] = positions_matrix

    # Log values of the posteriors
    logposteriors_matrix = sample[1]
    posteriors[:, 0, index] = logposteriors_matrix
    
# Sample matrices
m_samples = chains[:,0,burn_in:].reshape(((N_iter-burn_in)*n_walkers,1))
q_samples = chains[:,1,burn_in:].reshape(((N_iter-burn_in)*n_walkers,1))

# Posteriors matrix
post = np.column_stack((m_samples, q_samples))

#### We can use [*corner.py*](https://corner.readthedocs.io/en/latest/) to plot the chains and the posterior pdfs
Corner plots summarize the information about posterior pdfs and correlation between parameters in a simple triangular plot.

In [ ]:
import corner

labels = [r"$m$", r"$q$"]

figure = corner.corner(post, labels=labels, \
quantiles=[0.16, 0.5, 0.84], \
show_titles=True, title_kwargs={"fontsize": 14}, color="k", title_fmt=".2f", \
use_math_text = True, truths=[m_true, q_true])

figure.savefig("corner_plot.pdf", format="pdf")


#### Let's explore the chains in the "physical" space ($x$, $f(x)$)
Here we show how the couples of ($m$, $q$) stored in the chains behave in the plot $x$ *vs* $f(x)$.

In [ ]:
%%time
# Plot the data points and the target function
import matplotlib.pyplot as plt
plt.close()
plt.errorbar(x,y,yerr,fmt='o',color='r', label='Data points')
n_samples = m_samples.shape[0]
x_line = np.linspace(0,10,10)
for m, q in post[::20,:]:
    y_true = m*x_line+q
    plt.plot(x_line,y_true, color='b', alpha=0.01)
plt.plot(x_line, m_true*x_line+q_true, color='k', label='Target function')
plt.xlabel('$x$', fontsize=20)
plt.ylabel('$f(x)$', fontsize=20)
plt.legend(fontsize=20, loc=4)
plt.show()